In [0]:
!pip install tensorflow==1.13.2
!pip install keras==2.2.4
!pip install iterative-stratification

In [2]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#####################################################################
#                           Set Z                                   #
#####################################################################
# Classic Multi-label algorithms + Neural Networks and Embeddings   #
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

from keras_preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import StratifiedKFold, KFold
pd.set_option('max_colwidth',400)
from keras.preprocessing.sequence import pad_sequences
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, CuDNNLSTM, Bidirectional, Dense, \
    LSTM, Conv1D, MaxPooling1D, Dropout, concatenate, Flatten, add, Conv2D
from keras import initializers, regularizers, constraints
from keras import backend as K
from keras.engine import Layer
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import Input, Model
from keras.optimizers import Adam
from keras.models import Sequential, clone_model
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from utilities.preprocess import Preproccesor
import time
import numpy as np
from keras.models import model_from_json
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, make_scorer, fbeta_score, multilabel_confusion_matrix,\
                            average_precision_score, precision_score, recall_score
import nltk
import warnings

def average_precision_wrapper(y, y_pred, view):
    return average_precision_score(y, y_pred.toarray(),average=view)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [0]:
hamm_scorer = make_scorer(hamming_loss, greater_is_better=False)
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [5]:
nltk.download('wordnet')
nltk.download('stopwords')
X, yt, y = Preproccesor.load_multi_label_data(True) #yt has continuous data, y has binary
label_names = ["violence","directed_vs_generalized","gender","race","national_origin","disability","religion","sexual_orientation"]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
!wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip'
#!wget 'http://nlp.stanford.edu/data/glove.42B.300d.zip' 
import zipfile
with zipfile.ZipFile("/content/crawl-300d-2M.vec.zip","r") as zip_ref:
    zip_ref.extractall()
    print(zip_ref.filelist)
#with zipfile.ZipFile("/content/glove.42B.300d.zip","r") as zip_ref:
#    zip_ref.extractall()
#    print(zip_ref.filelist)

del zip_ref

--2020-05-27 06:32:42--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  26.3MB/s    in 56s     

2020-05-27 06:33:38 (25.9 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

[<ZipInfo filename='crawl-300d-2M.vec' compress_type=deflate filemode='-rw-r--r--' file_size=4514687127 compress_size=1523784963>]


In [0]:
!rm '/content/crawl-300d-2M.vec.zip'
#!rm '/content/glove.42B.300d.zip'

In [0]:
embedding_path1 = "/content/crawl-300d-2M.vec" #FastText
#embedding_path2 = "/content/glove.42B.300d.txt" #Glove 300d
embed_size = 300

In [0]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')
def build_matrix(embedding_path, tk, max_features):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path, encoding = "utf-8"))

    word_index = tk.word_index
    nb_words = max_features
    embedding_matrix = np.zeros((nb_words + 1, 300))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
def create_embedding_matrix(embed, tk, max_features):
    if embed == 1:
      return build_matrix(embedding_path1, tk, max_features)
    elif embed == 2:
      return build_matrix(embedding_path2, tk, max_features)
    else:
      return np.concatenate([build_matrix(embedding_path1, tk, max_features), build_matrix(embedding_path2, tk, max_features)], axis=-1)
  

In [0]:
n_fold = 10
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=7)

In [0]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                              K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [0]:
def my_hamming_loss(y_true, y_pred):
    print(y_true,y_pred)
    y_true=K.cast(y_true, dtype='float32')
    y_pred=K.cast(y_pred, dtype='float32')
    print(y_true,y_pred)
    hamming_loss(y_true,y_pred)
    
    return K.mean(diff, axis=-1)

In [0]:
#Binary Relevance
def build_model1(X_train, y_train, X_valid, y_valid, max_len, max_features, embed_size, embedding_matrix, lr=0.0, lr_d=0.0, spatial_dr=0.0, dense_units=128, conv_size=128, dr=0.2, patience=3, fold_id=1):
    file_path = f"best_model_fold_{fold_id}.hdf5"
    check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min")
    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    main_input = Input(shape=(max_len,), name='main_input')
    x = (Embedding(max_features + 1, embed_size*2, input_length=max_len, weights=[embedding_matrix], trainable=False))(main_input)
    x = SpatialDropout1D(0.4)(x)
    x = Bidirectional(LSTM(150, return_sequences=True))(x)
    x = Bidirectional(LSTM(150, return_sequences=True))(x)
    hidden = concatenate([
        Attention(max_len)(x),
        GlobalMaxPooling1D()(x),
    ])
    hidden = Dense(1024, activation='selu')(hidden)
    hidden = Dropout(0.4)(hidden)
    hidden = Dense(512, activation='selu')(hidden)
    hidden = Dropout(0.4)(hidden)
    hidden1 = Dense(128, activation='selu')(hidden)
    output_lay1 = Dense(8, activation='sigmoid')(hidden1)
    model = Model(inputs=[main_input], outputs=output_lay1)
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=['binary_accuracy'])
    from keras.utils import plot_model
    plot_model(model, to_file='model1.png')
    model2 = Model(inputs=[main_input], outputs=output_lay1)
    model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_valid, y_valid), verbose=1, callbacks=[early_stop, check_point])
    model2.load_weights(file_path)
    model2.compile(loss="binary_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=['binary_accuracy'])
    return model2

In [0]:
#Classifier Chains
def build_model2(X_train, y_train, X_valid, y_valid, max_len, max_features, embed_size, embedding_matrix, lr=0.0, lr_d=0.0, spatial_dr=0.0, dense_units=128, conv_size=128, dr=0.2, patience=3, fold_id=1):
    file_path = f"best_model_fold_{fold_id}.hdf5"
    check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1, save_best_only=True, mode="min")
    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    main_input = Input(shape=(max_len,), name='main_input')
    x = (Embedding(max_features + 1, embed_size*2, input_length=max_len, weights=[embedding_matrix], trainable=False))(main_input)
    x = SpatialDropout1D(0.5)(x)
    x = Bidirectional(LSTM(150, return_sequences=True))(x)
    x = Bidirectional(LSTM(150, return_sequences=True))(x)
    hidden = concatenate([
        Attention(max_len)(x),
        GlobalMaxPooling1D()(x),
    ])
    hidden = Dense(1024, activation='selu')(hidden)
    hidden = Dropout(0.5)(hidden)
    hidden = Dense(512, activation='selu')(hidden)
    hidden = Dropout(0.5)(hidden)
    hidden1 = Dense(128, activation='selu')(hidden)
    output_lay1 = Dense(1, activation='selu')(hidden1)
    hidden2 = concatenate([hidden1,output_lay1])
    output_lay2 = Dense(1, activation='selu')(hidden2)
    hidden3 = concatenate([hidden2,output_lay2])
    output_lay3 = Dense(1, activation='selu')(hidden3)
    hidden4 = concatenate([hidden3,output_lay3])
    output_lay4 = Dense(1, activation='selu')(hidden4)
    hidden5 = concatenate([hidden4,output_lay4])
    output_lay5 = Dense(1, activation='selu')(hidden5)
    hidden6 = concatenate([hidden5,output_lay5])
    output_lay6 = Dense(1, activation='selu')(hidden6)
    hidden7 = concatenate([hidden6,output_lay6])
    output_lay7 = Dense(1, activation='selu')(hidden7)
    hidden8 = concatenate([hidden7,output_lay7])
    output_lay8 = Dense(1, activation='selu')(hidden8)

    hidden_l = concatenate([output_lay1,output_lay2,output_lay3,output_lay4,output_lay5,output_lay6,
                                  output_lay7,output_lay8])
    hidden_l = Dropout(0.5)(hidden_l)
    output_layer = Dense(8, activation='sigmoid')(hidden_l)

    model = Model(inputs=[main_input], outputs=output_layer)
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=['binary_accuracy','categorical_accuracy'])
    from keras.utils import plot_model
    plot_model(model, to_file='model2.png')
    model2 = Model(inputs=[main_input], outputs=output_layer)
    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_valid, y_valid), verbose=1, callbacks=[early_stop, check_point])
    model2.load_weights(file_path)
    model2.compile(loss="binary_crossentropy", optimizer=Adam(lr=lr, decay=lr_d), metrics=['binary_accuracy','categorical_accuracy'])
    return model2

In [18]:
max_features = 50000
scores = {}
scores = {}
scores.setdefault('test_F1_example', [])
scores.setdefault('test_F1_macro', [])
scores.setdefault('test_F1_micro', [])
scores.setdefault('test_precision_example', [])
scores.setdefault('test_precision_macro', [])
scores.setdefault('test_precision_micro', [])
scores.setdefault('test_recall_example', [])
scores.setdefault('test_recall_macro', [])
scores.setdefault('test_recall_micro', [])
scores.setdefault('test_average_precision_macro', [])
scores.setdefault('test_average_precision_micro', [])
scores.setdefault('test_Accuracy', [])
scores.setdefault('test_Hamm', [])
cm = []
mskf = MultilabelStratifiedKFold(n_splits=10, random_state=0)
fold_n=0
save_ys = []
save_yt = []
max_len = 150
embed_size = 150
embma = 1
name = "Mixed"
for train_index, test_index in mskf.split(X, y):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    tk = Tokenizer(lower = True, filters='', num_words=max_features, oov_token = True)
    tk.fit_on_texts(X_train)
    train_tokenized = tk.texts_to_sequences(X_train)
    valid_tokenized = tk.texts_to_sequences(X_valid)
    X_train = pad_sequences(train_tokenized, maxlen=max_len)
    X_valid = pad_sequences(valid_tokenized, maxlen=max_len)
    embedding_matrix = create_embedding_matrix(embma, tk, max_features)

    model = build_model2(X_train, y_train, X_valid, y_valid, max_len, max_features, embed_size, embedding_matrix, lr=1e-3, lr_d=0, spatial_dr=0.1, dense_units=128, conv_size=128, dr=0.1, patience=10, fold_id=fold_n)
      
    fold_n = fold_n + 1
    yT = model.predict(X_valid)
    y_preds = []
    for yt in yT: #Don't do this if you throw them with continuous values
      yi = []
      for i in yt:
        if i>=0.5:
          yi.append(int(1))
        else:
          yi.append(int(0))
      y_preds.append(yi)
    y_preds = np.array(y_preds)
    scores['test_F1_example'].append(f1_score(y_valid, y_preds, average='samples'))
    scores['test_F1_macro'].append(f1_score(y_valid, y_preds, average='macro'))
    scores['test_F1_micro'].append(f1_score(y_valid, y_preds, average='micro'))
    scores['test_precision_example'].append(precision_score(y_valid, y_preds, average='samples'))
    scores['test_precision_macro'].append(precision_score(y_valid, y_preds, average='macro'))
    scores['test_precision_micro'].append(precision_score(y_valid, y_preds, average='micro'))
    scores['test_recall_example'].append(recall_score(y_valid, y_preds, average='samples'))
    scores['test_recall_macro'].append(recall_score(y_valid, y_preds, average='macro'))
    scores['test_recall_micro'].append(recall_score(y_valid, y_preds, average='micro'))
    scores['test_average_precision_macro'].append(average_precision_score(y_valid, y_preds, average='macro'))
    scores['test_average_precision_micro'].append(average_precision_score(y_valid, y_preds, average='micro'))
    scores['test_Accuracy'].append(accuracy_score(y_valid, y_preds))
    scores['test_Hamm'].append(hamming_loss(y_valid, y_preds))

f = open("../results/setE.txt", "a+")
f.write("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} \n".format(str(name)[:7],
                                              str('%.4f' % (sum(scores['test_F1_example'])/10)),
                                              str('%.4f' % (sum(scores['test_F1_macro'])/10)),
                                              str('%.4f' % (sum(scores['test_F1_micro']) / 10)),
                                              str('%.4f' % (sum(scores['test_precision_example']) / 10)),
                                              str('%.4f' % (sum(scores['test_precision_macro']) / 10)),
                                              str('%.4f' % (sum(scores['test_precision_micro']) / 10)),
                                              str('%.4f' % (sum(scores['test_recall_example']) / 10)),
                                              str('%.4f' % (sum(scores['test_recall_macro']) / 10)),
                                              str('%.4f' % (sum(scores['test_recall_micro']) / 10)),
                                              str('%.4f' % (sum(scores['test_average_precision_macro'])/10)),
                                              str('%.4f' % (sum(scores['test_average_precision_micro'])/10)),
                                              str('%.4f' % (sum(scores['test_Accuracy'])/10)),
                                              str('%.4f' % (sum(scores['test_Hamm'])/10))))
f.close()
print("{:<7} | {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} {:<7} \n".format(str(name)[:7],
                                            str('%.4f' % (sum(scores['test_F1_example'])/10)),
                                            str('%.4f' % (sum(scores['test_F1_macro'])/10)),
                                            str('%.4f' % (sum(scores['test_F1_micro']) / 10)),
                                            str('%.4f' % (sum(scores['test_precision_example']) / 10)),
                                            str('%.4f' % (sum(scores['test_precision_macro']) / 10)),
                                            str('%.4f' % (sum(scores['test_precision_micro']) / 10)),
                                            str('%.4f' % (sum(scores['test_recall_example']) / 10)),
                                            str('%.4f' % (sum(scores['test_recall_macro']) / 10)),
                                            str('%.4f' % (sum(scores['test_recall_micro']) / 10)),
                                            str('%.4f' % (sum(scores['test_average_precision_macro'])/10)),
                                            str('%.4f' % (sum(scores['test_average_precision_micro'])/10)),
                                            str('%.4f' % (sum(scores['test_Accuracy'])/10)),
                                            str('%.4f' % (sum(scores['test_Hamm'])/10))))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold 0 started at Wed May 27 08:44:16 2020
Train on 390 samples, validate on 43 samples
Epoch 1/50
390/390 [==============================] - 19s 50ms/step - loss: 0.6624 - binary_accuracy: 0.6567 - val_loss: 0.6217 - val_binary_accuracy: 0.7442

Epoch 00001: val_loss improved from inf to 0.62174, saving model to best_model_fold_0.hdf5
Epoch 2/50
390/390 [==============================] - 6s 16ms/step - loss: 0.6337 - binary_accuracy: 0.7103 - val_loss: 0.6052 - val_binary_accuracy: 0.7442

Epoch 00002: val_loss improved from 0.62174 to 0.60518, saving model to best_model_fold_0.hdf5
Epoch 3/50
390/390 [==============================] - 7s 17ms/step - loss: 0.6101 - binary_accuracy: 0.7340 - val_loss: 0.5882 - val_binary_accuracy: 0.7442

Epoch 00003: val_loss improved from 0.60518 to 0.58822, saving model to best_model_fold_0.hdf5
Epoch 4/50
390/390 [==============================] - 6s 17ms/step - loss: 0.6003 - binary_accuracy: 0.7362 - val_loss: 0.5742 - val_binary_accuracy: 0.7442

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 1 started at Wed May 27 08:53:22 2020
Train on 390 samples, validate on 43 samples
Epoch 1/50
390/390 [==============================] - 21s 54ms/step - loss: 0.6928 - binary_accuracy: 0.5551 - val_loss: 0.6400 - val_binary_accuracy: 0.6424

Epoch 00001: val_loss improved from inf to 0.64003, saving model to best_model_fold_1.hdf5
Epoch 2/50
390/390 [==============================] - 7s 17ms/step - loss: 0.6389 - binary_accuracy: 0.6510 - val_loss: 0.6094 - val_binary_accuracy: 0.6890

Epoch 00002: val_loss improved from 0.64003 to 0.60942, saving model to best_model_fold_1.hdf5
Epoch 3/50
390/390 [==============================] - 7s 17ms/step - loss: 0.6204 - binary_accuracy: 0.6769 - val_loss: 0.5983 - val_binary_accuracy: 0.6890

Epoch 00003: val_loss improved from 0.60942 to 0.59834, saving model to best_model_fold_1.hdf5
Epoch 4/50
390/390 [==============================] - 7s 17ms/step - loss: 0.5939 - binary_accuracy: 0.7138 - val_loss: 0.5745 - val_binary_accuracy: 0.7326

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 8 started at Wed May 27 09:58:03 2020
Train on 389 samples, validate on 44 samples
Epoch 1/50
389/389 [==============================] - 28s 71ms/step - loss: 0.6459 - binary_accuracy: 0.6228 - val_loss: 0.5600 - val_binary_accuracy: 0.7841

Epoch 00001: val_loss improved from inf to 0.55996, saving model to best_model_fold_8.hdf5
Epoch 2/50
389/389 [==============================] - 6s 17ms/step - loss: 0.5801 - binary_accuracy: 0.7298 - val_loss: 0.5240 - val_binary_accuracy: 0.7983

Epoch 00002: val_loss improved from 0.55996 to 0.52401, saving model to best_model_fold_8.hdf5
Epoch 3/50
389/389 [==============================] - 7s 17ms/step - loss: 0.5528 - binary_accuracy: 0.7423 - val_loss: 0.5080 - val_binary_accuracy: 0.7983

Epoch 00003: val_loss improved from 0.52401 to 0.50799, saving model to best_model_fold_8.hdf5
Epoch 4/50
389/389 [==============================] - 7s 17ms/step - loss: 0.5496 - binary_accuracy: 0.7677 - val_loss: 0.4990 - val_binary_accuracy: 0.7983

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 9 started at Wed May 27 10:07:37 2020
Train on 390 samples, validate on 43 samples
Epoch 1/50
390/390 [==============================] - 29s 74ms/step - loss: 0.6742 - binary_accuracy: 0.6099 - val_loss: 0.6307 - val_binary_accuracy: 0.7122

Epoch 00001: val_loss improved from inf to 0.63069, saving model to best_model_fold_9.hdf5
Epoch 2/50
390/390 [==============================] - 6s 16ms/step - loss: 0.6391 - binary_accuracy: 0.6606 - val_loss: 0.6045 - val_binary_accuracy: 0.7093

Epoch 00002: val_loss improved from 0.63069 to 0.60453, saving model to best_model_fold_9.hdf5
Epoch 3/50
390/390 [==============================] - 6s 16ms/step - loss: 0.6152 - binary_accuracy: 0.6837 - val_loss: 0.5816 - val_binary_accuracy: 0.7093

Epoch 00003: val_loss improved from 0.60453 to 0.58159, saving model to best_model_fold_9.hdf5
Epoch 4/50
390/390 [==============================] - 7s 17ms/step - loss: 0.5938 - binary_accuracy: 0.7000 - val_loss: 0.5601 - val_binary_accuracy: 0.7093

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
